<a href="https://colab.research.google.com/github/Juhyeon-creater/Sessac-project2/blob/main/YOLO%20Pose%20Normalization%20%26%20Preprocessing%20Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install ultralytics

In [ ]:
import os


# [경로 설정]
drive_root = '/content/drive/MyDrive/shared_googledrive(Sessac Final Project)/dataset/Hundred'#{Hundred(자세파일)} - output 도 이 파일에 생성
# 1. 원본 영상이 있는 폴더 (경로를 확인해주세요!)
video_folder_path = os.path.join(drive_root, 'hundred_video') #동영상들어있는파일 경로
# 2. 정규화된 CSV를 저장할 폴더
norm_folder_path = os.path.join(drive_root, 'norm')  # 저장할 폴더

os.makedirs(norm_folder_path, exist_ok=True)
video_exts = ('.mp4', '.avi', '.mov', '.mkv') # 처리할 영상 확장자


In [ ]:
from ultralytics import YOLO
import cv2
import pandas as pd
import numpy as np  # 정규화 계산을 위해 numpy 추가
import os
import glob # 파일 목록 비교를 위해 glob 추가
import time
from google.colab import drive

# 1. 드라이브 마운트 (최초 1회 실행)
try:
    drive.mount('/content/drive')
except:
    pass # 이미 마운트됨

# --- 2. 정규화 & 보간 함수 (Normalize & Interpolate) ---
# [최신 버전] 보간법이 포함된 정규화 함수입니다.
def normalize_keypoints_named(df):
    """
    [업그레이드 버전]
    1. 몸통 길이 기준으로 정규화 (Normalize)
    2. conf 낮은 값을 보간 (Interpolate)
    """
    df_norm = df.copy()

    # --- 1. 정규화 (Normalize) ---
    l_sh_x, l_sh_y = 'Left_Shoulder_x', 'Left_Shoulder_y'
    r_sh_x, r_sh_y = 'Right_Shoulder_x', 'Right_Shoulder_y'
    l_hip_x, l_hip_y = 'Left_Hip_x', 'Left_Hip_y'
    r_hip_x, r_hip_y = 'Right_Hip_x', 'Right_Hip_y'

    if l_sh_x not in df.columns or l_hip_x not in df.columns:
        print("   ⚠️ 정규화 필수 컬럼(어깨, 골반)이 없어 원본을 저장합니다.")
        return df

    neck_x = (df[l_sh_x] + df[r_sh_x]) / 2
    neck_y = (df[l_sh_y] + df[r_sh_y]) / 2
    pelvis_x = (df[l_hip_x] + df[r_hip_x]) / 2
    pelvis_y = (df[l_hip_y] + df[r_hip_y]) / 2

    torso_length = np.sqrt((neck_x - pelvis_x)**2 + (neck_y - pelvis_y)**2)
    torso_length = torso_length.replace(0, np.nan).fillna(1)

    body_parts = [
        'Nose', 'Left_Eye', 'Right_Eye', 'Left_Ear', 'Right_Ear',
        'Left_Shoulder', 'Right_Shoulder',
        'Left_Elbow', 'Right_Elbow', 'Left_Wrist', 'Right_Wrist',
        'Left_Hip', 'Right_Hip', 'Left_Knee', 'Right_Knee',
        'Left_Ankle', 'Right_Ankle'
    ]

    for part in body_parts:
        col_x = f'{part}_x'
        col_y = f'{part}_y'

        if col_x in df.columns and col_y in df.columns:
            df_norm[f'norm_{part}_x'] = (df[col_x] - pelvis_x) / torso_length
            df_norm[f'norm_{part}_y'] = (df[col_y] - pelvis_y) / torso_length

    # --- 2. 보간 (Interpolate) ---
    for part in body_parts:
        conf_col = f'{part}_conf'
        norm_x_col = f'norm_{part}_x'
        norm_y_col = f'norm_{part}_y'

        if conf_col in df_norm.columns and norm_x_col in df_norm.columns:
            low_conf_mask = df_norm[conf_col] < 0.5 # 0.5 미만 기준
            df_norm.loc[low_conf_mask, norm_x_col] = np.nan
            df_norm.loc[low_conf_mask, norm_y_col] = np.nan

    norm_cols = [col for col in df_norm.columns if col.startswith('norm_')]
    df_norm[norm_cols] = df_norm[norm_cols].interpolate(method='linear', limit_direction='both', limit=5)

    return df_norm

# --- 3. 메인 실행부 (YOLO + 정규화) ---

# --- [핵심 수정] 처리할 파일 목록 비교 ---
print("파일 목록 비교 중...")
all_videos = glob.glob(os.path.join(video_folder_path, "*")) # 모든 파일/폴더
all_video_files = [f for f in all_videos if f.lower().endswith(video_exts)] # 영상 파일만

# 2. 이미 'norm' 폴더에 있는 CSV 파일 목록 (이름만)
existing_norms = glob.glob(os.path.join(norm_folder_path, "norm_*.csv"))
# (예: 'norm_video1.csv' -> 'video1')
existing_norm_names = {os.path.basename(f).replace('norm_', '').replace('.csv', '') for f in existing_norms}

# 3. (비교) 처리해야 할 영상 목록 생성
videos_to_process = []
for video_path in all_video_files:
    # (예: '.../videos/video1.mp4' -> 'video1')
    video_name = os.path.splitext(os.path.basename(video_path))[0]
    if video_name not in existing_norm_names:
        videos_to_process.append(video_path)

if not videos_to_process:
    print("🎉 모든 영상이 이미 정규화되었습니다. 새로 처리할 파일이 없습니다.")
    exit() # 프로그램 종료
else:
    print(f"총 {len(all_video_files)}개 영상 중 {len(videos_to_process)}개를 새로 처리합니다...")
# --- [비교 끝] ---

# 모델 로드
print("⏳ YOLO 모델을 로드 중입니다...")
model = YOLO('yolo11x-pose.pt') # (모델 경로 확인)
print("✅ 모델 로드 완료!")

# 컬럼 이름 미리 정의 (COCO Keypoints 17개)
keypoint_names = [
    "Nose", "Left_Eye", "Right_Eye", "Left_Ear", "Right_Ear",
    "Left_Shoulder", "Right_Shoulder", "Left_Elbow", "Right_Elbow",
    "Left_Wrist", "Right_Wrist", "Left_Hip", "Right_Hip",
    "Left_Knee", "Right_Knee", "Left_Ankle", "Right_Ankle"
]

# [수정] videos_to_process 목록만 순회
total_files = len(videos_to_process)
for i, video_path in enumerate(videos_to_process):
    filename = os.path.basename(video_path)
    csv_filename = os.path.splitext(filename)[0] + '.csv'
    save_path = os.path.join(norm_folder_path, f"norm_{csv_filename}")

    print(f"\n[{i+1}/{total_files}] 분석 중... {filename}")
    start_time = time.time()

    cap = cv2.VideoCapture(video_path)

    # (영상 안 볼거면 show_video = False로 변경)
    show_video = False
    if show_video:
        # ... (이전과 동일한 cv2.namedWindow 코드) ...
        pass

    video_data = [] # 영상 1개의 모든 프레임 데이터를 담을 리스트
    frame_idx = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret: break

        results = model(frame, verbose=False, conf=0.3) # (신뢰도 0.3으로 낮춰서 더 많이 잡게 설정)

        if results[0].keypoints is not None and len(results[0].keypoints) > 0:
            person_kpts = results[0].keypoints.data[0].cpu().numpy()
            frame_data = {"Frame": frame_idx}

            for k, kp in enumerate(person_kpts):
                x, y, conf = kp
                name = keypoint_names[k]
                frame_data[f"{name}_x"] = round(float(x), 2)
                frame_data[f"{name}_y"] = round(float(y), 2)
                frame_data[f"{name}_conf"] = round(float(conf), 2)

            video_data.append(frame_data)

        # ... (show_video 관련 코드) ...

        frame_idx += 1
        if frame_idx % 100 == 0: print(".", end="", flush=True)

    cap.release()
    if show_video: cv2.destroyAllWindows()

    # --- [핵심 수정] 저장 직전에 정규화+보간 적용 ---
    if video_data:
        df_raw = pd.DataFrame(video_data)

        print(" ...정규화 및 보간 적용 중...")
        # [업그레이드된 함수 호출]
        df_normalized_interpolated = normalize_keypoints_named(df_raw)

        df_normalized_interpolated.to_csv(save_path, index=False)

        elapsed = time.time() - start_time
        print(f"\n   ✅ 정규화 완료 및 저장! ({elapsed:.1f}초) -> {save_path}")
    else:
        print(f"\n   ⚠️ 데이터 없음 -> {filename}")

print("\n🎉 모든 신규 작업 끝!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
파일 목록 비교 중...
총 74개 영상 중 73개를 새로 처리합니다...
⏳ YOLO 모델을 로드 중입니다...
✅ 모델 로드 완료!

[1/73] 분석 중... 필라테스_가산_A_Mat_Hundred_고급_actorP075_20220930_11.15.35_CAM_1.mp4
..... ...정규화 및 보간 적용 중...

   ✅ 정규화 완료 및 저장! (40.8초) -> /content/drive/MyDrive/shared_googledrive(Sessac Final Project)/dataset/Hundred/norm/norm_필라테스_가산_A_Mat_Hundred_고급_actorP075_20220930_11.15.35_CAM_1.csv

[2/73] 분석 중... 필라테스_가산_A_Mat_Hundred_고급_actorP075_20220930_11.17.02_CAM_1.mp4
..... ...정규화 및 보간 적용 중...

   ✅ 정규화 완료 및 저장! (42.3초) -> /content/drive/MyDrive/shared_googledrive(Sessac Final Project)/dataset/Hundred/norm/norm_필라테스_가산_A_Mat_Hundred_고급_actorP075_20220930_11.17.02_CAM_1.csv

[3/73] 분석 중... 필라테스_가산_A_Mat_Hundred_고급_actorP076_20220929_15.26.04_CAM_1.mp4
...... ...정규화 및 보간 적용 중...

   ✅ 정규화 완료 및 저장! (42.7초) -> /content/drive/MyDrive/share